In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.0'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-11-29 07:01:45--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-11-29 07:01:45 (6.59 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# from amazon read in data from S3 buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...| 2015-08-31|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with IR...|Home Entertainment|      

In [5]:
# Filter and create a new DataFrame
most_votes = df.filter('total_votes>=20')
most_votes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24284435|R1VCSDR38XTALJ|B00WA42GTC|     464704031|Samsung Blu-ray P...|Home Entertainment|          1|           33|         40|   N|                N|Wouldn't recommen...|This Samsung BD-J...| 2015-08-31|
|         US|    5445862|R1KMC6B2JRCZ3D|B00SMBF3NS|     742282603|VIZIO E24-C1 E Se...|Home Entertainment|      

In [6]:
# Filter and create a new DataFrame
helpful_reviews = most_votes.filter((df['helpful_votes'] / df['total_votes']) >= 0.5)
helpful_reviews.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24284435|R1VCSDR38XTALJ|B00WA42GTC|     464704031|Samsung Blu-ray P...|Home Entertainment|          1|           33|         40|   N|                N|Wouldn't recommen...|This Samsung BD-J...| 2015-08-31|
|         US|    5445862|R1KMC6B2JRCZ3D|B00SMBF3NS|     742282603|VIZIO E24-C1 E Se...|Home Entertainment|      

In [7]:
# Create a new DataFrame which retrieves all the rows where a review was written as part of the Vine program 
vine_reviews = helpful_reviews.filter('vine=="Y"')
vine_reviews.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51288086|R3LLICBKRHLRJC|B00USEZ7W4|     202667718|Seiki SE32HYT  LE...|Home Entertainment|          4|           17|         22|   Y|                N|Great picture, a ...|Very nice, sharp ...| 2015-08-27|
|         US|   50853411| RUD135S06DEVE|B00ZIGRX4W|     471919351|Samsung JS7000, S...|Home Entertainment|      

In [8]:
# Create a new DataFrame which retrieves all the rows where a review was written as part of the Vine program 
non_vine_reviews = helpful_reviews.filter('vine=="N"')
vine_reviews.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51288086|R3LLICBKRHLRJC|B00USEZ7W4|     202667718|Seiki SE32HYT  LE...|Home Entertainment|          4|           17|         22|   Y|                N|Great picture, a ...|Very nice, sharp ...| 2015-08-27|
|         US|   50853411| RUD135S06DEVE|B00ZIGRX4W|     471919351|Samsung JS7000, S...|Home Entertainment|      

In [14]:
# Determine the total number of vine reviews
vine_reviews = df.filter('vine=="Y"').count()
vine_reviews

2106

In [15]:
# Determine the total number of non vine reviews
not_vine_reviews = df.filter('vine=="N"').count()
not_vine_reviews

703783

In [17]:
# Determine the number of 5-star vine reviews 
vine_five_star = df.filter('vine=="Y"').filter('star_rating==5').count()
vine_five_star

854

In [18]:
# Determine the number of 5-star non vine reviews 
not_vine_five_star = df.filter('vine=="N"').filter('star_rating==5').count()
not_vine_five_star

373134

In [19]:
# Determine the percentage of 5-star reviews for vine reviews
vine_percent = df.filter('vine=="Y"').filter('star_rating==5').count() / df.filter('vine=="Y"').count()
vine_percent

0.40550807217473883

In [20]:
# Determine the percentage of 5-star reviews for non vine reviews
not_vine_percent = df.filter('vine=="N"').filter('star_rating==5').count() / df.filter('vine=="N"').count()
not_vine_percent

0.5301833093439313